# Résolution Pb42 de la collection Hock-Schittkowski

Classification : QQR-T1-10

Number of variables : n = 4

Number of constraints : 2 (2 equalities)

# Modèle

$\left\{  
\begin{array}{l l l l} 
\min\ f(x) \\ 
\text{s.c.}\\
x_1-2 = 0\\
x_3^2+x_4^2-2= 0
\end{array} \right.$

where $f : x \longmapsto (x_1-1)^2+(x_2-2)^2+(x_3-3)^2+(x_4-4)^2$



Point de départ : $x_0 = (1,1,1,1)$

Solution attendue : $x^* = (2,2,0.6\sqrt 2, 0.8\sqrt 2)$

Valeur de la fonction objectif à la solution : $f(x^*) =28-10\sqrt 2$

In [6]:
include("../src/enlsip_functions.jl")

enlsip

In [2]:
n = 4
m = 4
nb_eq = 2
nb_constraints = 2

2

In [3]:
res42 = ResidualsEval(0)

function (res42::ResidualsEval)(x::Vector{Float64}, rx::Vector{Float64}, J::Matrix{Float64})

    # Evaluate the residuals
    if abs(res42.ctrl) == 1
        rx[:] = [x[1] - 1.0;
        x[2] - 2.0;
        x[3] - 3.0;
        x[4] - 4.0]

    # The jacobian is computed analytically
    elseif res42.ctrl == 2
        res42.ctrl = 0
    end
    return
end

cons42 = ConstraintsEval(0)

function (cons42::ConstraintsEval)(x::Vector{Float64}, cx::Vector{Float64}, A::Matrix{Float64})

    # Evaluate the constraints
    if abs(cons42.ctrl) == 1
        cx[:] = [x[1] - 2.0;
                 x[3]*x[3] + x[4]*x[4] - 2.0]

    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif cons42.ctrl == 2
        cons42.ctrl = 0
    end
    return
end

In [8]:
include("../src/enlsip_functions.jl")
x0 = [1.0;1.0;1.0;1.0]
enlsip42 = enlsip(x0,res42,cons42,n,m,nb_eq,nb_constraints,MAX_ITER=8,scaling=true)

Itération 0

Itération 1

Itération 2

Itération 3

Itération 4
[newton_search_direction] b = [0.0, -0.037711125603396164]
[newton_search_direction] p1 = [0.0, 0.03771112560339617]

Itération 5
[newton_search_direction] b = [0.0, -0.0007326570340443724]
[newton_search_direction] p1 = [0.0, 0.0007326570340443725]

Itération 6
[newton_search_direction] b = [0.0, -7.414775682720425e-5]
[newton_search_direction] p1 = [0.0, 7.414775682720425e-5]

Itération 7
[newton_search_direction] b = [0.0, -2.3791991705512516e-5]
[newton_search_direction] p1 = [0.0, 2.379199170551252e-5]

Itération 8
[newton_search_direction] b = [0.0, -2.3791991705512516e-5]
[newton_search_direction] p1 = [0.0, 2.379199170551252e-5]

****************************************
*                                      *
*          ENLSIP-JULIA-0.3.0          *
*                                      *
****************************************

Starting point : [1.0, 1.0, 1.0, 1.0]

Number of equality constraints   : 2
Number o

ENLSIP([2.0, 2.0, 0.8399138715307083, 1.1378101702207915], 13.85810270409699)

In [5]:
x1 = enlsip42.sol[1]
x2 = enlsip42.sol[2]
x3 = enlsip42.sol[3]
x4 = enlsip42.sol[4]

println("Solution trouvée avec ENLSIP-Julia :")
@printf "x_jul = (%.9e, %.9e, %.9e, %.9e)\n" x1 x2 x3 x4
@printf "f(x_jul) = %.9e\n\n" enlsip42.obj_value 

println("\nValeurs théoriques visées :")
@printf "x_opt = (%.9e, %.9e, %.9e, %.9e)\n" 2.0 2.0 0.6*sqrt(2) 0.8*sqrt(2)
@printf "f(x_opt) = %.9e" 28 - 10*sqrt(2)

Solution trouvée avec ENLSIP-Julia :
x_jul = (2.000000000e+00, 2.000000000e+00, 8.485281362e-01, 1.131370851e+00)
f(x_jul) = 1.385786438e+01


Valeurs théoriques visées :
x_opt = (2.000000000e+00, 2.000000000e+00, 8.485281374e-01, 1.131370850e+00)
f(x_opt) = 1.385786438e+01

In [6]:
using JuMP, Ipopt

In [7]:
model = Model(with_optimizer(Ipopt.Optimizer))

@variable(model, x1, start = 1.0)
@variable(model, x2, start = 1.0)
@variable(model, x3, start = 1.0)
@variable(model, x4, start = 1.0)

x4

In [8]:
f(x1,x2,x3,x4) = (x1 - 1)^2 + (x2 - 2)^2 + (x3 - 3)^2 + (x4 - 4)^2
JuMP.register(model, :f, 4, f, autodiff=true)

In [9]:
@NLconstraint(model, c1, x3^2 + x4^2 - 2.0 == 0)
@constraint(model, x1 - 2.0 == 0)

x1 = 2.0

In [10]:
@NLobjective(model, Min, f(x1,x2,x3,x4))
JuMP.optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [11]:
@printf "x_ipopt = (%.9e, %.9e, %.9e, %.9e)\n" JuMP.value(x1) JuMP.value(x2) JuMP.value(x3) JuMP.value(x4)

x_ipopt = (2.000000000e+00, 1.999999999e+00, 8.485281373e-01, 1.131370850e+00)
